In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten

# Đường dẫn tới dữ liệu huấn luyện
train_dir = 'Black2/train'

label = ['Apple', 'Lemon', 'Mango', 'Orage', 'Bell pepper', 'Pear', 'Plums', 'Strawberries']

# Thiết lập ImageDataGenerator cho tập huấn luyện
train_data_gen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_data_gen.flow_from_directory(
    train_dir,
    target_size=(200, 200),
    class_mode='categorical',
    shuffle=False  # Đảm bảo không xáo trộn dữ liệu để có thể khớp giữa đặc trưng và nhãn
)




Found 142 images belonging to 8 classes.


In [2]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))


e:\Python_code\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [3]:
# Xây dựng mô hình

# Trích xuất đặc trưng từ lớp Dense trước lớp cuối cùng
feature_extractor = Model(inputs=model.inputs, outputs=model.layers[-2].output)

# Dự đoán đặc trưng trên tập dữ liệu huấn luyện
train_features = feature_extractor.predict(train_generator, batch_size=32)

# Lưu các đặc trưng đã trích xuất nếu cần thiết
np.save('train_features.npy', train_features)

# Lưu mô hình trích xuất đặc trưng
feature_extractor.save('feature_extractor_model.h5')

e:\Python_code\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 665ms/step


In [4]:
import os
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
def find_similar_images():
    # Load đặc trưng của ảnh đầu vào
    input_image_path = selected_image_path.get()
    input_image = image.load_img(input_image_path, target_size=(200, 200))
    input_image = image.img_to_array(input_image)
    input_image = np.expand_dims(input_image, axis=0)
    input_feature = feature_extractor.predict(input_image)

    # Tính toán độ tương đồng giữa ảnh đầu vào và tất cả các ảnh trong tập huấn luyện
    similarities = []
    for train_feature in train_features:
        similarity = np.dot(input_feature.flatten(), train_feature.flatten())
        similarities.append(similarity)

    # Lựa chọn ra 3 ảnh có độ tương đồng cao nhất
    top_indices = np.argsort(similarities)[-3:][::-1]

    # Hiển thị các ảnh tương đồng trong giao diện
    similar_images = []
    for index in top_indices:
        similar_image_path = train_generator.filepaths[index]
        similar_image = Image.open(similar_image_path)
        similar_image = similar_image.resize((200, 200))
        similar_images.append(ImageTk.PhotoImage(similar_image))

    # Hiển thị các ảnh tương đồng trong giao diện
    for i, similar_img in enumerate(similar_images):
        similar_image_label = tk.Label(app, image=similar_img)
        similar_image_label.image = similar_img
        similar_image_label.pack(side=tk.LEFT, padx=5)

app = tk.Tk()
app.title("Fruit Classifier")

last_directory = "/"

screen_width = app.winfo_screenwidth()
screen_height = app.winfo_screenheight()

content_width = 850
content_height = 850

x = (screen_width - content_width) // 2
y = (screen_height - content_height) // 2

app.geometry(f"{content_width}x{content_height}+{x}+{y}")

def open_file_dialog():
    global last_directory
    file_path = filedialog.askopenfilename(initialdir=last_directory, title="Select Image", 
                                           filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        last_directory = os.path.dirname(file_path)
        display_image(file_path)

def display_image(file_path):
    global img
    image = Image.open(file_path)
    image = image.resize((200, 200))  
    img = ImageTk.PhotoImage(image)
    image_label.config(image=img)
    image_label.image = img
    selected_image_path.set(file_path)

title_label = tk.Label(app, text="Fruit Classifier", font=("Helvetica", 20, "bold"))  
title_label.pack(side=tk.TOP, pady=15)

image_label = tk.Label(app)
image_label.pack()

selected_image_path = tk.StringVar()

result_label = tk.Label(app, text="", font=("Helvetica", 12))  
result_label.pack(pady=15)

button_frame = tk.Frame(app)
button_frame.pack(side=tk.BOTTOM, pady=15)

open_button = tk.Button(button_frame, text="Open Image", command=open_file_dialog)  
open_button.pack(side=tk.LEFT, padx=5)

find_similar_button = tk.Button(button_frame, text="Find Similar Images", command=find_similar_images)  
find_similar_button.pack(side=tk.LEFT, padx=5)

app.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
